# 処理まとめ

In [1]:
import numpy as np
import pandas as pd

## index
- 関数
  - [「記号-数字」の組み合わせをソート](#「記号-数字」の組み合わせをソート)
  - [数字xx桁にして返す関数](#数字xx桁にして返す関数)
  - [価格表記に変換](#価格表記に変換)
- おまけ
  - [列のハッシュ化](#列のハッシュ化)

---

## 「記号-数字」の組み合わせをソート

In [2]:
# sample date
df = pd.DataFrame({ 'A' : [1, 2, 3, 4, 5, 6, 7, 8, 9,],
                    'B' : ['A-10', 'B-3', 'A-1', 'C-2', 'A-2', 'B-2', 'B-1', 'C-1', np.nan],})

In [3]:
df

,A,B
0,1,A-10
1,2,B-3
2,3,A-1
3,4,C-2
4,5,A-2
5,6,B-2
6,7,B-1
7,8,C-1
8,9,NaN


In [4]:
def sort_symbol(df, before_col="B", sort_col='sort_col'):
    """ 2020/09/10
    
    アルファベット、数字の組み合わせがある値をソートする。
    そのままソートすると数字が文字列の為上手くソートできない
    ハイフンで切り分けて、数字はnp.intにキャストした
    
    ExSample:
    
        before  after
        --------------
        A-10          <- NaNは削除
        B-3     A-1
        A-1     A-10
        C-2     B-3
        NaN     C-2
    
    params:
    	df(DataFrame)	: 処理してるデータ
        before_col(str) : 処理対象column name
        sort_col(str)   : sortしたcolumn name
        
    return:
        sort_col：ソートした結果返す
    """
    
    # NaN(float)を0に置換してnp.intにキャストする.floatでソートできない為
    df[sort_col] = df[before_col].fillna('0')
    
    df['symbol'] = df.apply(
        lambda row: f"{row[sort_col]}".split('-')[0] if '-' in row[sort_col] else row[sort_col], axis=1)
    
    df['num'] = df.apply(
        lambda row: f"{row[sort_col]}".split('-')[1] if '-' in row[sort_col] else 0, axis=1).astype(np.int)
    
    df = df.sort_values(['symbol', 'num'])
    
    df = df.replace('0','')
    
    return df

In [5]:
df = sort_symbol(df, before_col="B", sort_col='sort_col')
df

,A,B,sort_col,symbol,num
8,9,NaN,,,0
2,3,A-1,A-1,A,1
4,5,A-2,A-2,A,2
0,1,A-10,A-10,A,10
6,7,B-1,B-1,B,1
5,6,B-2,B-2,B,2
1,2,B-3,B-3,B,3
7,8,C-1,C-1,C,1
3,4,C-2,C-2,C,2


### 最終表示 sortできた

In [6]:
df[['A', 'B', 'sort_col']]

,A,B,sort_col
8,9,NaN,
2,3,A-1,A-1
4,5,A-2,A-2
0,1,A-10,A-10
6,7,B-1,B-1
5,6,B-2,B-2
1,2,B-3,B-3
7,8,C-1,C-1
3,4,C-2,C-2


---

## 数字xx桁にして返す関数

In [7]:
# sample data
df = pd.DataFrame({ 'A' : 1.,
                    'G' : ['1000', '', '123', '1234',]})

In [8]:
def rep_digit_num(df, before_col='code', after_col='L_code', digit=5):
    """
	digit桁にした値で返す
	先頭を0埋めにし、(np.str)でキャストしておく.
    csv出力した時に先頭の0が消滅するのを防ぐ
    
    ExSample:
    	123  ==>> 00123
        1234 ==>> 01234
    
    params:
    	df(DataFrame)	: 処理してるデータ
        before_col(str) : 処理対象col
        after_col(str)  : 処理したcol
        digit(int)		: repしたい桁数(default: 5)
    
    return:
    	after_col(np.str)digit桁にした値を返す
    """
    
    df[after_col] = df.apply(lambda row: f"{row[before_col]}".zfill(digit), axis=1).astype(np.str)    
    return df

In [9]:
rep_digit_num(df, before_col='G', after_col='G_rep', digit=5)

,A,G,G_rep
0,1.0,1000,01000
1,1.0,,00000
2,1.0,123,00123
3,1.0,1234,01234


***

## 価格表記に変換

In [10]:
df = pd.DataFrame({ 'A' : 1.,
                    'G' : ['1000', np.nan, '123', '1234', '12345', 123456]})

In [11]:
df

,A,G
0,1.0,1000
1,1.0,NaN
2,1.0,123
3,1.0,1234
4,1.0,12345
5,1.0,123456


In [12]:
def convert_to_price(df, before_col='brfore_col', after_col="price"):
    """
	値段表記に変換する
    
    ExSample:
    	123.0  ==>> 123
        1234.0 ==>> 1,234
        NaN	   ==>> 0,000
    
    params:
    	df(DataFrame)	: 処理してるデータ
        before_col(str) : 処理対象col
        after_col(str)  : 処理したcol
        
    return:
    	after_col(np.str)digit桁にした値を返す
    """
    # NaN(float)を0に置換してnp.intにキャストする.
    df[before_col] = df[before_col].fillna('0').astype(np.int)
    
    df[after_col] = df.apply(lambda row: f"{row[before_col]:,}", axis=1) \
        .replace(r'^0', '0,000', regex=True).str.split('.', expand=True)[0]
    
    return df

## 結果

In [13]:
convert_to_price(df, before_col='G', after_col='price')

,A,G,price
0,1.0,1000,"1,000"
1,1.0,0,"0,000"
2,1.0,123,123
3,1.0,1234,"1,234"
4,1.0,12345,"12,345"
5,1.0,123456,"123,456"


---

## 列のハッシュ化

In [14]:
df = pd.DataFrame({ 'A' : 1.,
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                    'D' : np.array([3] * 4,dtype='int32'),
                    'E' : pd.Categorical(["test","train","test","train"]),
                    'F' : 'foo',
                    'G' : ['1000', np.nan, '123', '1234',]})

In [15]:
import hashlib

df['hash_G'] = df['G'].astype('str').apply(lambda n: hashlib.sha256(n.encode()).hexdigest())

In [16]:
df

,A,B,C,D,E,F,G,hash_G
0,1.0,2013-01-02,1.0,3,test,foo,1000,40510175845988f13f6162ed8526f0b09f73384467fa85...
1,1.0,2013-01-02,1.0,3,train,foo,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...
2,1.0,2013-01-02,1.0,3,test,foo,123,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...
3,1.0,2013-01-02,1.0,3,train,foo,1234,03ac674216f3e15c761ee1a5e255f067953623c8b388b4...
